In [56]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import itertools
import random
import string
import io
import warnings

In [4]:
character_list = list(string.ascii_lowercase) + [".","-"," ","+"]
character_lookup = dict(zip(character_list, range(len(character_list))))
max_length = 10
num_characters = len(character_lookup)

In [5]:
model = keras.models.load_model("model.h5")

In [60]:
def generate_name(model, character_lookup, max_length, temperature = 1):
    def choose_next_char(preds, character_lookup, temperature = 1):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="divide by zero encountered in log")
            preds = np.log(preds)/temperature
        exp_preds = np.exp(preds)
        preds = exp_preds/sum(np.exp(preds))

        next_index = np.argmax(np.random.multinomial(1,preds))
        next_character = list(character_lookup.keys())[next_index]
        return next_character
    def characters_to_matrix(character_data):
        character_data = [[character_lookup[chr] for chr in c] for c in character_data]
        padded_character_data = keras.utils.pad_sequences(character_data, maxlen = max_length)
        text_matrix = keras.utils.to_categorical(padded_character_data, num_classes = num_characters)
        return text_matrix

    in_progress_name = []
    next_letter = ""
  
    while next_letter != "+" and len(in_progress_name) < 30:
        previous_letters_data = characters_to_matrix([in_progress_name])
        next_letter_probabilities = model.predict(previous_letters_data, verbose = 0)[0]
        next_letter = choose_next_char(next_letter_probabilities, character_lookup, temperature)
        if next_letter != "+":
            in_progress_name = in_progress_name + [next_letter]
    
    raw_name = ''.join(in_progress_name)
    capitalized_name = string.capwords(raw_name) 
  
    return capitalized_name

def generate_many_names(model, character_lookup, max_length, temperature = 1, n = 10):
    names = [generate_name(model, character_lookup, max_length, temperature) for i in range(n)]
    return names

In [61]:
generate_many_names(model, character_lookup, max_length, n = 20, temperature = 0.5)

['Mesa',
 'Jack',
 'Molly',
 'Bella',
 'Missy',
 'Marley',
 'Rocky',
 'Samper',
 'Bella',
 'Max',
 'Milo',
 'Bella',
 'Samson',
 'Buttercut',
 'Ollie',
 'Marley',
 'Roger',
 'Nora',
 'Nella',
 'Odin']